In [1]:
import json, pandas as pd
import numpy as np

##### 감성 분석 파일에서 시작

In [20]:
train = pd.read_csv('../data/train_sentiment.csv')
dev = pd.read_csv('../data/dev_sentiment.csv')

In [21]:
train.head()

,user_id,user_profile_age_range,user_profile_name,user_profile_residence,user_profile_accepted_food,user_profile_accepted_movies,user_profile_accepted_music,user_profile_rejected_music,user_profile_gender,user_profile_accepted_celebrities,...,topic,wday,sentence_index,sentence,goal_topic,goal_type,knowledge,recdial,sentiment_star,sentiment_score
0,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,NaN,NaN,0,Who is the leading actor of the movie Left Rig...,Left Right Love Destiny,Q&A,"Cecilia Cheung, Stars, Left Right Love Destiny",0,1,0.309721
1,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,NaN,NaN,1,It's Cecilia Cheung,Left Right Love Destiny,Q&A,"Cecilia Cheung, Stars, Left Right Love Destiny",0,3,0.316815
2,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,NaN,NaN,2,You even know that. Excellent!,Left Right Love Destiny,Q&A,NaN,0,5,0.936480
3,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,NaN,NaN,3,Thank you for your praise. She's a Chinese ido...,Cecilia Cheung,Chat about stars,"Cecilia Cheung, Intro, Chinese dreamgirl",0,5,0.682868
4,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,NaN,NaN,4,Yes my idol is quite unusual.,Cecilia Cheung,Chat about stars,NaN,0,5,0.609620


### 봇이 시작할때가 있고, 사람이 시작할 때가 있다. 어떻게 구분해야 할까?
#### 맨 처음 대화를 할 때, 대화의 goal type을 보자. (user_id당 첫번째 대화의 goal type) -> 6개 밖에 없다.

In [22]:
# idx 0번째에는 sentence
# idx 1번째에는 goal_type
# user_id번째의 index에 두가지 정보가 저장되어 있음
# first_start_list[example_user_id][0 or 1] 과 같이 사용가능

first_start_list = []
user_list = []

for idx in range(len(train)):
    if train['user_id'].iloc[idx] not in user_list:
        first_start_list.append([train['sentence'].iloc[idx], train['goal_type'].iloc[idx]])
        user_list.append(train['user_id'].iloc[idx])

In [33]:
# dev에도 적용
first_start_list = []
user_list = []

for idx in range(len(dev)):
    if dev['user_id'].iloc[idx] not in user_list:
        first_start_list.append([dev['sentence'].iloc[idx], dev['goal_type'].iloc[idx]])
        user_list.append(dev['user_id'].iloc[idx])

#### 전체 goal_type의 종류를 확인해보자

In [23]:
type_set = set([x[1] for x in first_start_list])
type_set

{'Ask about date',
 'Ask about time',
 'Ask about weather',
 'Greetings',
 'Music on demand',
 'Q&A'}

In [34]:
# dev에도 적용

type_set = set([x[1] for x in first_start_list])
type_set

{'Ask about date',
 'Ask about time',
 'Ask about weather',
 'Greetings',
 'Music on demand',
 'Q&A'}

#### Greetings를 제외하고 다 유저가 먼저 물어보는 경우인듯

In [35]:
# 맨 첫번째 대화가 Greetings인 user_id와 첫문장이 나온다
for idx in range(len(first_start_list)):
    if first_start_list[idx][1] == 'Greetings':
        print(idx, first_start_list[idx][0])

3 Hello.
5 Good morning, Ms. Jingjia Han.
6 Hello.
13 Good evening. What are you doing?
15 Good evening. How are you doing?
19 Good evening!
25 What are you doing, Haiqi Ma.
28 Good evening!
31 Good morning. How are you doing?
32 Hello, do you want to talk to me about your life?
33 Good morning. How are you?
36 Hi, how are you?
37 How have you been doing?
41 Good evening!
43 Hello, Ms. Naxia Yu!
45 Hello! Mr. Qihua Hao, how have you been doing?
47 Good afternoon, dear friend. How are you doing?
52 Hello, how are you doing?
53 Good morning, Chaoyu Hu. How are you doing recently?
59 Good evening. How are you doing recently?
60 Hi, how are you doing?
63 Good evening, Xinbin Liu. How are you doing?
64 Good morning.
65 Good afternoon!
66 Good evening, Miss Fangyang Liu. How are you?
68 Good evening. How are you doing?
69 Good afternoon. How was your day?
70 Good evening. How are you doing?
72 Good evening, Lizhou Xu.
74 Hello, Mr.Liyu Wu!
76 Good afternoon, Chaohang Yang.
78 Good evening, M

In [25]:
# 확인하기, 100번까지 확인해본 결과 이론이 맞음
# 맨 user_id의 첫 번째 대화가 greeting이면 봇 시작/아니면 사람(user)의 요구로 대화를 시작해서서 사람 시작임
# 특히 18번은 먼저 시작한 쪽에서 추천함 -> 봇
check_user_id = 21
for sentence in train[train['user_id']==check_user_id]['sentence']:
    print(sentence)

What's the date today?
Today is September 12, 2018.
Great. You know a lot.
Today's Yang Mi's birthday. Do you like her?
Of course, she is the goddess in my heart.
Haha, She has won the Most Popular Actress of The Magnolia Award of Shanghai TV Festival. She is very popular. And she is also good at acting. She has won Best Actress of Houston International Film Festival.
Yeah, she is good at acting.
Have you seen her The Breakup Guru? It has a good script and full of laughters.
What kind of movie is this? How about its public opinion?
It's a comedy and boasts good reputation.
Sounds good. I'll watch it when I'm free.
I hope it will give you a different experience
OK, I have something else to do.Talk to you next time.
OK, see ya!


In [36]:
check_user_id = 21
for sentence in dev[dev['user_id']==check_user_id]['sentence']:
    print(sentence)

I want to know how tall is Leehom Wang. I feel like he is very tall.
He is very tall. He's 5 feet 10 inches.
You are so smart. Thank you.
Speaking of Leehom Wang, he is a top star in Taiwan. He's a man of extraordinary abilities and fine appearance.
You're right. He's perfect in the entertainment industry.
He has also won the MTV Asia Awards for Favorite Artist Taiwan and the 15th Global Chinese Music Awards for Best Male Singer in Hong Kong and Taiwan Region.
This is the proof of his achievements.
It shows that he is an excellent singer. I recommend you to listen to his Chinese Zodiac. It's a catchy song with 12 Chinese zodiac signs appearing in order in the lyrics. The cheerful melody and catchy lyrics has won it vast popularity.
Yes, this song is classic. I can recite the Chinese zodiac signs in order now because of this song.
Which means it's a great song.
Indeed. It reminds me of a lot of things.
Do you want to me play Chinese zodiac?
Yes.
OK, now playing Chinese zodiac for you.
O

맨 첫번째 대화가 GREETING일때는 무조건 봇이고 나머지는 사람이 필요로 의해서 말을 거는것 같다.

In [26]:
# is_bot_talk_first 카테고리 생성성
# bot이 대화를 시작한 대화면 1, 아니면 0이다 (true / false)
data = []
for idx in range(len(train)):
    if first_start_list[train.iloc[idx]['user_id']][1] == 'Greetings':
        data.append(1)
    else:
        data.append(0)

train['is_bot_talk_first'] = data

In [37]:
data = []
for idx in range(len(dev)):
    if first_start_list[dev.iloc[idx]['user_id']][1] == 'Greetings':
        data.append(1)
    else:
        data.append(0)

dev['is_bot_talk_first'] = data

#### is_user : 사용자(추천을 받는 사람)인지 봇(추천을 하는 친구)인지
##### 1: 사용자, 0: 봇

In [30]:
is_user = []
for idx in range(len(train)):
    if train.iloc[idx]['is_bot_talk_first'] == 0: # 유저가 먼저 대화 시작
        if train.iloc[idx]['sentence_index'] % 2 == 0:
            is_user.append(1) # 유저
        else:
            is_user.append(0) # 봇
    else: # 봇이 먼저 대화 시작
        if train.iloc[idx]['sentence_index'] % 2 == 0:
            is_user.append(0) # 봇
        else:
            is_user.append(1) # 유저

train['is_user'] = is_user


In [31]:
train

,user_id,user_profile_age_range,user_profile_name,user_profile_residence,user_profile_accepted_food,user_profile_accepted_movies,user_profile_accepted_music,user_profile_rejected_music,user_profile_gender,user_profile_accepted_celebrities,...,sentence_index,sentence,goal_topic,goal_type,knowledge,recdial,sentiment_star,sentiment_score,is_bot_talk_first,is_user
0,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,0,Who is the leading actor of the movie Left Rig...,Left Right Love Destiny,Q&A,"Cecilia Cheung, Stars, Left Right Love Destiny",0,1,0.309721,0,1
1,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,1,It's Cecilia Cheung,Left Right Love Destiny,Q&A,"Cecilia Cheung, Stars, Left Right Love Destiny",0,3,0.316815,0,0
2,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,2,You even know that. Excellent!,Left Right Love Destiny,Q&A,NaN,0,5,0.936480,0,1
3,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,3,Thank you for your praise. She's a Chinese ido...,Cecilia Cheung,Chat about stars,"Cecilia Cheung, Intro, Chinese dreamgirl",0,5,0.682868,0,0
4,0,Under 18 years old,Fangyang Liu,Qingdao,Jiaozi Stuffed with Mackerel,"['Left Right Love Destiny', 'Hot Summer Days',...",['Once'],['Its Time'],Female,"['Cecilia Cheung', 'Kris Wu', 'Nicholas Tse']",...,4,Yes my idol is quite unusual.,Cecilia Cheung,Chat about stars,NaN,0,5,0.609620,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87281,5677,Under 18 years old,Fangshan Liu,Urumqi,Peanuts with Black Vinegar,['Left Right Love Destiny'],"['The Moon Represents My Heart', 'Run Toward t...","['Silence is Gold', 'A Man Of Intention']",Female,"['Cecilia Cheung', 'Leslie Cheung', 'Nicholas ...",...,13,It's on now.,Together We Move On,Play music,NaN,0,4,0.280243,0,0
87282,5677,Under 18 years old,Fangshan Liu,Urumqi,Peanuts with Black Vinegar,['Left Right Love Destiny'],"['The Moon Represents My Heart', 'Run Toward t...","['Silence is Gold', 'A Man Of Intention']",Female,"['Cecilia Cheung', 'Leslie Cheung', 'Nicholas ...",...,14,It's really nice.,Together We Move On,Play music,NaN,0,5,0.516064,0,1
87283,5677,Under 18 years old,Fangshan Liu,Urumqi,Peanuts with Black Vinegar,['Left Right Love Destiny'],"['The Moon Represents My Heart', 'Run Toward t...","['Silence is Gold', 'A Man Of Intention']",Female,"['Cecilia Cheung', 'Leslie Cheung', 'Nicholas ...",...,15,I'm glad you like it.,Together We Move On,Play music,NaN,0,5,0.522069,0,0
87284,5677,Under 18 years old,Fangshan Liu,Urumqi,Peanuts with Black Vinegar,['Left Right Love Destiny'],"['The Moon Represents My Heart', 'Run Toward t...","['Silence is Gold', 'A Man Of Intention']",Female,"['Cecilia Cheung', 'Leslie Cheung', 'Nicholas ...",...,16,"It's time to have a dinner, goodbye.",Say goodbye,Say goodbye,NaN,0,5,0.481954,0,1


In [38]:
is_user = []
for idx in range(len(dev)):
    if dev.iloc[idx]['is_bot_talk_first'] == 0: # 유저가 먼저 대화 시작
        if dev.iloc[idx]['sentence_index'] % 2 == 0:
            is_user.append(1) # 유저
        else:
            is_user.append(0) # 봇
    else: # 봇이 먼저 대화 시작
        if dev.iloc[idx]['sentence_index'] % 2 == 0:
            is_user.append(0) # 봇
        else:
            is_user.append(1) # 유저

dev['is_user'] = is_user


In [39]:
dev

,user_id,user_profile_age_range,user_profile_name,user_profile_residence,user_profile_accepted_food,user_profile_accepted_movies,user_profile_accepted_music,user_profile_rejected_music,user_profile_gender,user_profile_accepted_celebrities,...,sentence_index,sentence,goal_topic,goal_type,knowledge,recdial,sentiment_star,sentiment_score,is_bot_talk_first,is_user
0,0,Over 50 years old,Jacky Cheung's news,Jiujiang,['Wanzhou Roasted Fish'],['Marinated Fish'],"['The Best Voice', 'If Its Not Love', 'The Cre...","['Blessing', 'Hard to Go Back']",Male,['Jacky Cheung'],...,0,What's the weather like today?,Ask about weather,Ask about weather,"Jiujiang, 2018-10-4, Sunny with north winds. T...",0,3,0.328311,0,1
1,0,Over 50 years old,Jacky Cheung's news,Jiujiang,['Wanzhou Roasted Fish'],['Marinated Fish'],"['The Best Voice', 'If Its Not Love', 'The Cre...","['Blessing', 'Hard to Go Back']",Male,['Jacky Cheung'],...,1,It's sunny with north winds in Jiujiang today....,Ask about weather,Ask about weather,"Jiujiang, 2018-10-4, Sunny with north winds. T...",0,3,0.392084,0,0
2,0,Over 50 years old,Jacky Cheung's news,Jiujiang,['Wanzhou Roasted Fish'],['Marinated Fish'],"['The Best Voice', 'If Its Not Love', 'The Cre...","['Blessing', 'Hard to Go Back']",Male,['Jacky Cheung'],...,2,That's great. That's detailed.,Ask about weather,Ask about weather,NaN,0,4,0.379832,0,1
3,0,Over 50 years old,Jacky Cheung's news,Jiujiang,['Wanzhou Roasted Fish'],['Marinated Fish'],"['The Best Voice', 'If Its Not Love', 'The Cre...","['Blessing', 'Hard to Go Back']",Male,['Jacky Cheung'],...,3,It's nice weather to rat Marinated Fish.,Marinated Fish,Food recommendation,"Sunny with north winds. The high will be 27 ℃,...",1,4,0.507903,0,0
4,0,Over 50 years old,Jacky Cheung's news,Jiujiang,['Wanzhou Roasted Fish'],['Marinated Fish'],"['The Best Voice', 'If Its Not Love', 'The Cre...","['Blessing', 'Hard to Go Back']",Male,['Jacky Cheung'],...,4,"Yo, you even know my favorite Marinated Fish.",Marinated Fish,Food recommendation,NaN,1,5,0.932400,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12652,810,Under 18 years old,Bo Huang's news; Jay Chou's news,Luoyang,NaN,['Mapo Tofu'],"['How Are You', 'Rain', 'Dear Child', 'Magicia...","['Self Directed Act', 'A Branch of Plum Blosso...",Female,"['Bo Huang', 'Jay Chou']",...,3,Yeah. It will turn from sunny to cloudy with n...,Weather notification,Weather notification,"Luoyang, 2018-11-21, Sunny early and mostly cl...",0,2,0.282932,0,0
12653,810,Under 18 years old,Bo Huang's news; Jay Chou's news,Luoyang,NaN,['Mapo Tofu'],"['How Are You', 'Rain', 'Dear Child', 'Magicia...","['Self Directed Act', 'A Branch of Plum Blosso...",Female,"['Bo Huang', 'Jay Chou']",...,4,The weather forecast is quite accurate. I like...,Weather notification,Weather notification,NaN,0,4,0.639092,0,1
12654,810,Under 18 years old,Bo Huang's news; Jay Chou's news,Luoyang,NaN,['Mapo Tofu'],"['How Are You', 'Rain', 'Dear Child', 'Magicia...","['Self Directed Act', 'A Branch of Plum Blosso...",Female,"['Bo Huang', 'Jay Chou']",...,5,Thank you.,Weather notification,Weather notification,NaN,0,5,0.604014,0,0
12655,810,Under 18 years old,Bo Huang's news; Jay Chou's news,Luoyang,NaN,['Mapo Tofu'],"['How Are You', 'Rain', 'Dear Child', 'Magicia...","['Self Directed Act', 'A Branch of Plum Blosso...",Female,"['Bo Huang', 'Jay Chou']",...,6,I'm going to have a rest. Talk later!,Say goodbye,Say goodbye,NaN,0,3,0.272668,0,1


카테고리를 생성한 csv를 저장하자 (sep = 누가시작하는지 알았으니 유저랑 봇을 분리한다는 뜻)

In [32]:
train.to_csv('../data/train_sent_sep.csv')

In [ ]:
dev.to_csv('../data/dev_sent_sep.csv')